In [ ]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("klein2111/scannet-3d")


# import shutil
# shutil.copytree(src_path, dst_path)
# print("Copied dataset to:", dst_path)

100%|██████████| 3.57G/3.57G [05:34<00:00, 11.4MB/s]

Extracting files...


Path to dataset files: /home/avishka/.cache/kagglehub/datasets/klein2111/scannet-3d/versions/1


In [9]:
# ...existing code...
import torch
import numpy as np
from pathlib import Path

pth_path = Path("data/scannet_3d/train/scene0000_00_vh_clean_2.pth")
obj = torch.load(pth_path, map_location="cpu")
print(f"Loaded: {pth_path}\nType: {type(obj)}")

for i in range(len(obj)):
    print(f"Key: {i}, Shape: {obj[i].shape}, min: {obj[i].min()}, max: {obj[i].max()}")

Loaded: data/scannet_3d/train/scene0000_00_vh_clean_2.pth
Type: <class 'tuple'>
Key: 0, Shape: (81369, 3), min: -0.016569163650274277, max: 8.740397453308105
Key: 1, Shape: (81369, 3), min: -0.9921568632125854, max: 1.0
Key: 2, Shape: (81369,), min: 0.0, max: 255.0


In [ ]:
python scannet_download.py -o data --id scene0711_00